In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from fastai.basics import *
from fastai.callback.all import *
from fastai.vision.all import *

In [ ]:
df = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')

In [ ]:
df.head()

In [ ]:
def get_x(df):
    return '../input/cassava-leaf-disease-classification/train_images/' + str(df["image_id"]) 
def get_y(df):
    s = df['label'] 
    return s

In [ ]:
df.info()

In [ ]:
df['label'].unique()

In [ ]:
df['label'] = df['label'].apply(lambda x: str(x))

In [ ]:
def get_dls(size):
    dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                   get_x=get_x,
                   get_y=get_y,
                   item_tfms=Resize(460),
                   batch_tfms=[*aug_transforms(size=size, min_scale=0.75),
                               Normalize.from_stats(*imagenet_stats)])
    return dblock.dataloaders(df, bs=32)

dls = get_dls(128)

In [ ]:
dls.show_batch(nrows=1, ncols=3)

In [ ]:
if not os.path.exists('/root/.cache/torch/hub/checkpoints/'):
        os.makedirs('/root/.cache/torch/hub/checkpoints/')
!cp '../input/resnet50/resnet50.pth' '/root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth'

In [ ]:

learner = cnn_learner(dls, resnet50, loss_func=LabelSmoothingCrossEntropy(), metrics=accuracy
                      , cbs=MixUp()
                     )
#loss_func=LabelSmoothingCrossEntropy()
#learner = cnn_learner(dls, resnet18, pretrained=False, metrics=partial(accuracy_multi, thresh=0.2)).to_fp16() 

In [ ]:
learner.lr_find()

In [ ]:
learner.fine_tune(8, base_lr=2e-2, freeze_epochs=5) 

In [ ]:
#learner.save('presize')

In [ ]:
#learner = learner.load('presize')

Now for some progressive resizing

In [ ]:
learner.dls = get_dls(360) # progressive resizing
learner.lr_find()

In [ ]:
learner.fine_tune(7, base_lr=2e-4, freeze_epochs=2) 

In [ ]:
# learner.dls = get_dls(400) # progressive resizing
# learner.lr_find()

In [ ]:
# learner.fine_tune(7, base_lr=1e-4, freeze_epochs=2) 

In [ ]:
learner.recorder.plot_loss()

Lets apply some test time augmentation

In [ ]:
learner.to_native_fp32()
preds, targs = learner.tta()
print(accuracy(preds, targs))

In [ ]:
path = Path('../input/cassava-leaf-disease-classification')
submission_df = pd.read_csv(path/'sample_submission.csv')
submission_df

In [ ]:
test_data_path = submission_df['image_id'].apply(lambda x: path/'test_images'/x)
tst_dl = learner.dls.test_dl(test_data_path)
predictions = learner.tta(dl = tst_dl, n=10, beta=0)

submission_df['label'] = np.argmax(predictions[0],axis=1)
submission_df

In [ ]:
submission_df.to_csv('submission.csv',index=False)